In [1]:
# load data and class label
import numpy as np
import pandas as pd
from pandas import DataFrame


dflabels = pd.read_csv('project_class_labels_45.csv',index_col='Unnamed: 0')
processeddf = pd.read_csv('project_data_down_45.csv',index_col='Unnamed: 0')

In [2]:
# check to see if there is any feature (i.e. column) has all zero values so we will delete them
removedAllZeroColdf = processeddf.loc[:, (processeddf != 0).any(axis=0)]
removedAllZeroColdf.shape

(1485, 20301)

In [3]:
# Feature Selection - Variance Threshold 

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=500000)
reduced = sel.fit_transform(removedAllZeroColdf)
reduceddf = DataFrame(reduced)
reduced.shape

(1485, 6636)

In [4]:
# data scaling
# method 1 : standardization
from sklearn.preprocessing import StandardScaler

stdscaler = StandardScaler()
stdscalerfit = stdscaler.fit_transform(removedAllZeroColdf)

stddf = DataFrame(stdscalerfit)
stddf.head()

,0,1,2,3,4,5,6,7,8,9,...,20291,20292,20293,20294,20295,20296,20297,20298,20299,20300
0,-0.192484,-0.755906,0.446054,-0.593338,-0.633973,-0.038358,-0.363468,-0.659568,-0.100470,-0.104769,...,-0.754288,-0.163153,0.630883,-0.294233,-0.597323,-1.256714,-1.302928,-0.436749,-0.386928,-0.110039
1,-0.192484,-0.638706,-0.720695,-0.229807,1.717049,-0.038358,-0.593566,-0.659568,-0.100470,-0.141879,...,-1.219220,-0.858947,-0.597955,-0.303946,-0.718719,-0.951909,-0.669963,-0.965714,0.493859,-0.176391
2,-0.192484,-0.719361,-0.624723,-0.810700,1.421606,-0.038358,-0.223596,0.248409,-0.100470,-0.120959,...,-0.982288,-0.875488,-0.513379,-0.288079,-0.787315,-1.166562,-0.067199,-1.252016,0.876467,-0.052508
3,-0.192484,-0.525673,-0.569783,-0.575637,-0.875379,-0.038358,-0.700962,2.190311,-0.100470,-0.147639,...,1.406053,0.620178,0.097046,-0.295408,-0.093643,-1.259139,-0.906240,-0.183397,0.955710,3.459447
4,2.521522,-0.712481,-0.671541,-0.761510,4.970730,-0.038358,-0.905740,-0.659568,0.612594,-0.147639,...,-1.220881,-1.397227,-0.589230,-0.260070,-0.788476,-1.083691,-1.070452,-1.122174,-0.186059,-0.176391


In [5]:
# split 80% training set; 20% testing set
from sklearn.model_selection import train_test_split

trainData, testData, trainLabel, testLabel = train_test_split(reduceddf, dflabels, test_size=0.20)

In [6]:
# decision tree

from sklearn import tree
from sklearn.metrics import accuracy_score
# measure training time
import time

entrophy_clf = tree.DecisionTreeClassifier(criterion='entropy',
            min_samples_leaf=1,min_samples_split=2, 
            min_weight_fraction_leaf=0.0)
start = time.time()
entrophy_clf = entrophy_clf.fit(trainData, trainLabel)
end = time.time()
print("decision tree-entrophy training time: ", end - start)

pred = entrophy_clf.predict(testData)
accuracy = accuracy_score(testLabel, pred)
print("Accuracy Score (entrophy decision tree):", accuracy)

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(testLabel, pred)
print(confusion_matrix)

# classficiation report
from sklearn.metrics import classification_report
print(classification_report(testLabel, pred))

decision tree-entrophy training time:  42.34432935714722
Accuracy Score (entrophy decision tree): 0.717171717172
[[5 0 0 ..., 0 0 0]
 [0 7 0 ..., 1 0 0]
 [0 1 8 ..., 0 1 0]
 ..., 
 [0 0 1 ..., 6 2 0]
 [0 0 0 ..., 0 5 0]
 [0 0 0 ..., 0 0 6]]
             precision    recall  f1-score   support

        ACC       0.50      0.83      0.62         6
       BLCA       0.64      0.78      0.70         9
       BRCA       0.80      0.73      0.76        11
       CESC       0.83      0.56      0.67         9
       CHOL       0.50      0.40      0.44        10
       COAD       0.73      0.57      0.64        14
       DLBC       1.00      1.00      1.00         9
       ESCA       0.75      0.75      0.75         8
        GBM       1.00      0.78      0.88         9
       HNSC       0.64      0.70      0.67        10
       KICH       0.58      0.85      0.69        13
       KIRC       0.78      0.50      0.61        14
       KIRP       1.00      0.83      0.91         6
       LAML     